# PBL Lab 4: portable EMG recording lab

In this lab you are introduced to the measurement and analysis of electrical activity generated by muscles.

Outline:
* [1. Prepare the Raspberry Pi Zero ](#Ch1)
    * [1.1 Battery Header and Raspberry Pi](#Ch11)
    * [1.2 Install required libraries](#Ch12)
    * [1.3 Connect the two boards (AD8232 & ADS1115)](#Ch13) 
* [2. Controlling EMG with Python code](#Ch2)
* [3. Creating an Interface to visualize the signals.](#Ch3)
* [4. EMG or ECG measurements](#Ch4)
    * [4.1 Sensor placement ](#Ch41)
    * [4.2 Saving data to file](#Ch42)
  


### 1. Prepare the Raspberry Pi Zero <a class="anchor" id="Ch1"></a>

##### 1.1 Battery Header and Raspberry Pi <a class="anchor" id="Ch11"></a>
1. Connect the battery header to your Raspberry Pi
* Connect the battery to the header board, move the wires out of the way of the connector pins
* Make sure the switch on the end of the board is __OFF__ before connecting 
* Add the spacers and screws to fix the two boards together

<div> <img src="images/UPS_HAT.jpg" width="600"> </div>

2. Start up your Raspberry Pi by switching the battery to __ON__

3. Go to the main menu and open the __Raspberry Pi Configuration__ tool

4. Select the __Interfaces__ tab and ensure that the __I2C__ is __enabled__

5. __Reboot__ your Raspberry Pi


##### 1.2 Install required libraries <a class="anchor" id="Ch12"></a>

1. Python Installation of ADS1x15 Library

We will use a board designed by Adafruit, it requires specific libraries to connect to your Raspbery Pi. First, you will need to install the Adafruit_Blinka library, it provides the CircuitPython support in Python. Open a terminal on your Raspberry Pi. From your command line run:

        >>sudo pip3 install Adafruit-Blinka

Second, we also need the more specific library that comes with the board (ADS1115) that we will use. From your command line run the following command:

        >>sudo pip3 install adafruit-circuitpython-ads1x15


#### 1.3 Connect the two boards (AD8232 & ADS1115) <a class="anchor" id="Ch13"></a>

1. Shut down the Raspberry Pi. 

In your lab-box you find two boards. The red board is the ECG board that uses a chip (AD8232) to measure electrical activity of a muscle. 
* The AD8232 is designed to extract, amplify, and filter small biopotential signals in the presence of noisy conditions, such as those created by motion or remote electrode placement.ECGs can be extremely noisy, the AD8232 Single Lead Heart Rate Monitor acts as an op amp to help obtain a clear signal. (_note: This is NOT a medical device and is not intended to be used as such or as an accessory to such nor diagnose or treat any conditions._) ; 
* The blue board (ADS1115) is the Analog-to-Digital Converter (ADC) which convert the analogue output from the sensor to a digital signal. As you may recall, the Raspberry Pi can only read digital signals, so we need this analog-to-digital converter here to read out the signals from the AD8232. 

2. Connect the boards as shown in the image.

3. Switch the battery to 'ON' to switch on the RP.

<div>
<img src="images/KT2502_Lab4_EMGcircuit_bb.png" width="600">
</div>


### 2. Controlling EMG with Python code <a class="anchor" id="Ch2"></a>

1. Open a Python editor on the RP to start a new Python3 script
2. Save the script in a new folder: _Documents/Lab4/EMG_test.py_

3. To demonstrate that the device functions, we will start with a simple example that read the ADC channel values at a single instance. 

In [ ]:
# import the necessary modules and initialize the I2C bus:
import board
import busio
i2c = busio.I2C(board.SCL, board.SDA)

# import the module for the board you are using. For the ADS1115, use:
import adafruit_ads1x15.ads1115 as ADS

# Note that we are renaming each import to ADS for convenience.
# The final import needed is for the ADS1x15 library's version of AnalogIn:
from adafruit_ads1x15.analog_in import AnalogIn

# Create the ADC object, for the ADS1115.
ads = ADS.ADS1115(i2c)

Now let's see how to get values from the board. You can use these boards in either single ended or differential mode. The usage for the two modes are slightly different.

In principle, the ADS1115 board can export the difference between two channels (e.g. between A0 and A1) (differential mode) or the individual values from the channels, which is called the __Single Ended Mode__. Since the AS8232 board already takes care of exporting the difference between the channels, we will only use the single ended mode. For single ended mode we use AnalogIn to create the analog input channel, providing the ADC object and the pin to which the signal is attached. Here, we use pin 0. 

4. Add the following to your code:

In [ ]:
chan = AnalogIn(ads, ADS.P0)

To set up additional channels, use the same syntax but provide a different pin.

5. Now you can read the raw value and voltage of the channel using either the the value or voltage property. Add the following line to print your outcomes in the REPL.


In [ ]:
 print(chan.value, chan.voltage)

6. Now run the code to check the result!

### 3. Creating an Interface to visualize the signals. <a class="anchor" id="Ch3"></a>

Now that we have seen the first outputs of the device, let's create a simple interface with which we can start and stop the measurement and see a running data plot while capturing. The interface should have the following:
* graph with running data


This blog helps you with some inital code: https://learn.sparkfun.com/tutorials/graph-sensor-data-with-python-and-matplotlib/speeding-up-the-plot-animation

Optional:
* Try to visualize several lines in one figure in seperate (sub)plots
* Add a start and stop button


In [ ]:
## Fill out your own code on the Raspberry Pi


### 4. EMG or ECG measurements <a class="anchor" id="Ch4"></a>

#### 4.1 Sensor placement <a class="anchor" id="Ch41"></a>
To test whether the outcome signal plotted in the previous is correct, test it by measuring an ECG or measure the EMG of a single muscle. 

###### __ECG__
For ECG use this configuration to test:
<div> <img src="images/ECG_placement.jpg" width="200"> </div>


##### _EMG_
In general for the EMG placement, [seniam.org](http://seniam.org/) is a good website to learn about best sensor locations. The two measurement electrodes (red and yellow) should be between the innervation zone and the tendinous insertion of the muscle, along the longitudinal midline of the muscle. They should be 2 to 4 cm apart and over the belly of the muscle. The reference electrode (green) should be on a bony part of the body with very little muscle activity.
<div> <img src="images/EMG_biceps.png" width="200"> </div>

Determine how to best place the electrodes to obtain the best result.

#### 4.2 Saving data to file <a class="anchor" id="Ch42"></a>
In the previous sections you have extracted and plotted live EMG data using the ADS1115 library. However, we cannot analyse data if this isn't saved. You are now going to create a script such that the output values and time stamps are written to a csv file.

1. Make sure that you save a new Python3 file: logging_EMG_data.py

The first thing we need when we want to save experimental data is a timestamp for each keypoint. And, while we are on it, it would be great if the data are automatically saved with a filename that holds the date and time.

2. Therefore, we need to import __datetime__ from the library __datetime__

To explore what __datetime__ does, you can run the following example.

In [ ]:
from datetime import datetime

x = datetime.now()
print(x)

The date contains year, month, day, hour, minute, second, and microsecond. The __datetime__ module has many methods to return information about the date object.

| Directive |	Description |
| --- | --- |
| %a	| Weekday, short version	Wed	|
| %A	| Weekday, full version	Wednesday	| 
| %w	| Weekday as a number 0-6, 0 is Sunday	3| 	
| %d	| Day of month 01-31	31	| 
| %b	| Month name, short version	Dec | 	
| %B	| Month name, full version	December | 	
| %m	| Month as a number 01-12	12	| 
| %y	| Year, short version, without century	18	 | 
| %Y	| Year, full version	2018	| 
| %H	| Hour 00-23	17	| 
| %I	| Hour 00-12	05	| 
| %p	| AM/PM	PM	| 
| %M	| Minute 00-59	41	| 
| %S	| Second 00-59	08	| 
| %f	| Microsecond 000000-999999	548513	| 
| %z	| UTC offset	+0100	| 
| %Z	| Timezone	CST	| 
| %j	| Day number of year 001-366	365	| 
| %U	| Week number of year, Sunday as the first day of week, 00-53	52	| 
| %W	| Week number of year, Monday as the first day of week, 00-53	52	| 
| %c	| Local version of date and time	Mon Dec 31 17:41:00 2018	| 
| %C	| Century	20	| 
| %x	| Local version of date	12/31/18	| 
| %X	| Local version of time	17:41:00	| 
| %%	| A % character	%	| 
| %G	| ISO 8601 year	2018	| 
| %u	| ISO 8601 weekday (1-7)	1 | 	
| %V	| ISO 8601 weeknumber (01-53)	01	 |
_(source: https://www.w3schools.com/python/python_datetime.asp )_

3. Create a code to extract a string similar to: _20220404-114200_ (year,month,day - hour,minute,second)

In [ ]:
from datetime import datetime 

# Enter your own code here!


4. Use the code you just wrote in the _logging_EMG_data.py_ script to create a filename _data/output_yourdatestring.csv_. Don't forget to also import the __datetime__ library at the start of your script.

You will need the __datetime__ module again later on, but for now let's focus on creating a file in which we can log the data.

5. In Lecture 3, you have learnt how to open a file with a specific filename. Use this to open a file in which we can _write_ with the filename you created in (4). Place this code also in your script.

Instead of opening a text file like we did in Lecture 3 we now want to create a csv file. There is a special Python module we can use: __csv__

6. import the csv module in _logging_EMG_camera.py_

csv.writer class is used to insert data to the CSV file. This class returns a writer object which is responsible for converting the user’s data into a delimited string. A csvfile object should be opened with _newline=''_ otherwise newline characters inside the quoted fields will not be interpreted correctly. Therefore, we need to slightly adjust the code we put in (5)

7. add _newline=''_ within the brackets of the line in which you open the csv file. 

csv.writer class provides two methods for writing to CSV. They are __writerow()__ and __writerows()__. writerow() writes a single row at a time. writerow(fields) is used to write multiple rows at a time. Below is an example of how to write data into a csv file

In [ ]:
# CSV Example

import csv 
    
# field names 
fields = ['Name', 'Course', 'Year', 'Grade'] 
    
# data rows of csv file 
rows = [ ['Nikhil', 'KT2502', '2022', '9.0'], 
         ['Sanchit', 'KT2501', '2021', '7.1'], 
         ['Aditya', 'KT2502', '2022', '9.3'], 
         ['Sagar', 'KT2502', '2022', '9.5'], 
         ['Prateek', 'KT2502', '2022', '7.8'], 
         ['Sahil', 'KT2502', '2022', '9.1']] 


    
# name of csv file 
filename = "course_records.csv"
    
# writing to csv file 
csvfile = open(filename, 'w',newline='') 
    
# creating a csv writer object 
csvwriter = csv.writer(csvfile) 
        
# writing the fields 
csvwriter.writerow(fields) 
        
# writing the data rows 
csvwriter.writerows(rows)
    
csvfile.close()

This file is saved in the same folder as this python notebook.

7. Use this example to create a csv writer object. Think about how you would like to export the keypoints, what will be your headers? Create this header (fields) and write it to the csv file.

The rows will be filled with the EMG data while you are measuring. 

8. As an example, the following code creates a single row

In [ ]:
row = []
row.append(value1)
row.append(value2)
           
# add date to row
row.insert(0,now) # enters the date to as first entry of the row

   
9. Write the data array that you constructed to the csv file.

10. Finally, make sure that when the program stops, you close the csv file.

In [ ]:
## Fill out your own code on the Raspberry Pi
